In [1]:
!ls

sample_data


In [2]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shujatali123","key":"76be1feee77195c0715b2ccb5ed0d093"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d salader/dogs-vs-cats --force

 96% 521M/545M [00:15<00:00, 29.7MB/s]
100% 545M/545M [00:15<00:00, 36.9MB/s]


In [0]:
!unzip dogs-vs-cats.zip

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


# Data pre-processing and data augmentation


In [0]:
IMG_WIDTH, IMG_HEIGHT = 150, 150
TRAIN_DATA_DIR = 'train'
VALIDATION_DATA_DIR = 'test'
MODEL_WEIGHTS_FILE = 'vgg16-xfer-tuned-weights.h5'
NB_TRAIN_SAMPLES = 20000
NB_VALIDATION_SAMPLES = 800
NB_EPOCH = 15

In [0]:
train_datagen = ImageDataGenerator(   # this is the augmentation configuration we will use for training
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)# this is the augmentation configuration we will use for testing:
                                                 # only rescaling

In [0]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        TRAIN_DATA_DIR,# this is the target directory
        target_size=(IMG_WIDTH, IMG_HEIGHT),# all images will be resized to 150x150
        batch_size=50,
        class_mode='categorical')


Found 20000 images belonging to 2 classes.


In [0]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

### this is a similar generator, for validation data

In [0]:
validation_generator = test_datagen.flow_from_directory(
        VALIDATION_DATA_DIR,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=50,
        class_mode='categorical')

Found 5000 images belonging to 2 classes.


# Using the pre-trained model VGG-16 

In [11]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step


In [12]:
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [0]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

In [14]:
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

## Adding layers to Pre trained Model

In [15]:
input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(input=input, output=x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                524352    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 15,239,170
Trainable params: 524,482
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy', #compiling model
              optimizer='rmsprop',
              metrics=['accuracy'])


In [0]:
model.fit_generator(
        train_generator,
        callbacks = callbacks,
        steps_per_epoch=100,
        nb_epoch=NB_EPOCH,
        validation_data=validation_generator,
        validation_steps=100,
        verbose=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=100, validation_data=<keras_pre..., validation_steps=100, verbose=2, epochs=15)`
  if __name__ == '__main__':


Epoch 1/15
 - 55s - loss: 0.8814 - acc: 0.7460 - val_loss: 0.2685 - val_acc: 0.8842
Epoch 2/15
 - 51s - loss: 0.3905 - acc: 0.8276 - val_loss: 0.2408 - val_acc: 0.8948
Epoch 3/15
 - 51s - loss: 0.3201 - acc: 0.8602 - val_loss: 0.2561 - val_acc: 0.8908
Epoch 4/15
 - 51s - loss: 0.2999 - acc: 0.8708 - val_loss: 0.3607 - val_acc: 0.8392
Epoch 5/15
 - 52s - loss: 0.2829 - acc: 0.8758 - val_loss: 0.2371 - val_acc: 0.8964
Epoch 6/15
 - 51s - loss: 0.2756 - acc: 0.8780 - val_loss: 0.2636 - val_acc: 0.8878
Epoch 7/15
 - 50s - loss: 0.2751 - acc: 0.8816 - val_loss: 0.2114 - val_acc: 0.9074
Epoch 8/15
 - 52s - loss: 0.2743 - acc: 0.8816 - val_loss: 0.2379 - val_acc: 0.9004
Epoch 9/15
 - 52s - loss: 0.2559 - acc: 0.8952 - val_loss: 0.2230 - val_acc: 0.9080
Epoch 10/15
 - 50s - loss: 0.2546 - acc: 0.8914 - val_loss: 0.4830 - val_acc: 0.8200
Epoch 11/15
 - 51s - loss: 0.2490 - acc: 0.8958 - val_loss: 0.2162 - val_acc: 0.9086
Epoch 12/15
 - 51s - loss: 0.2542 - acc: 0.8888 - val_loss: 0.2198 - val_a

In [0]:
# callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
# history = model.fit_generator(
#         train_generator,
#         callbacks = callbacks,

#         samples_per_epoch=NB_TRAIN_SAMPLES,#2000  (NB_TRAIN_SAMPLES = 2000)   /32(batch_size=32) =1epoch
#         nb_epoch=NB_EPOCH,#50
#         validation_data=validation_generator,
#         nb_val_samples=NB_VALIDATION_SAMPLES)

# Predictions

In [0]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [0]:
import numpy as np
img_path = 'doggy.jpeg'
img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
features[[0]]

array([[0., 1.]], dtype=float32)

In [0]:
y_prob = model.predict(x) 
y_classes = y_prob.argmax(axis=-1)
y_classes

array([1])

In [0]:
import numpy as np
img_path = 'cat.jpg'
img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
features

array([[1., 0.]], dtype=float32)

In [0]:
y_prob = model.predict(x) 
y_classes = y_prob.argmax(axis=-1)
y_classes

array([0])